In [42]:
#dataset set up
from torch.utils.data import Dataset
from torchvision.transforms import RandomCrop, ToTensor
from PIL import Image
import pandas as pd
import pickle
import os
from torchvision import transforms




class ProductsDataset(Dataset):
    def __init__(self) -> None:
        super().__init__()
        self.data_products = pd.read_csv("/home/jupyter/clean_products_data.csv", index_col=0, lineterminator='\n')
        self.data_products.reset_index(drop=False, inplace=True)
        self.data_images = pd.read_csv("/home/jupyter/Images.csv",index_col=0)

        self.encoder_dict = self.init_encoder()
        self.decoder_dict = self.init_decoder()
        

        #self.min_image_size = self._min_image_size()

        #self.pil_to_tensor = ToTensor()
        #self.resize = RandomCrop(140,140)

        self.data_example = self._data_transform()

        self.transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.CenterCrop(224),
        #transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])


    def init_encoder(self):
        encoding_dict = {category:code for code, category in enumerate(set(self.data_products.category.apply(lambda x: str(x.split("/")[0]).strip())))}
        #print("length dict encoder "+ str(len(encoding_dict)))
        file = open("encoder.pkl", 'wb')
        pickle.dump(encoding_dict, file)
        file.close()
        return encoding_dict

    def init_decoder(self):
        decoding_dict = encoding_dict = {code:category for category, code in self.encoder_dict.items()}
        
        file = open("decoder.pkl", 'wb')
        pickle.dump(encoding_dict, file)
        file.close()
        return decoding_dict

    def encoding(self):
        self.data_products["label"] = self.data_products.category.apply(lambda x: self.encoder_dict.get(str(x.split("/")[0]).strip()))

    def _data_transform(self):
        self.encoding()

        #self.data_products.rename({self.data_products.id:"product_id"}, inplace=True)
        self.data_products['product_id'] = self.data_products['id']
        self.data_images['images_id'] = self.data_images['id']
        data = pd.merge( left=self.data_images, right=self.data_products[["id", "label"]], how="left", left_on='product_id', right_on="id")
        data = data[["images_id", "label"]]
        data = data.loc[(data.label==0) | (data.label == 1) ]#
        #data.sort_values('label', inplace=True)
        data.reset_index(drop=True, inplace=True)
        return data

    def _min_image_size(self):
        images = os.listdir("/home/jupyter/images/")
        check_min_size = []
        for im in images:
            im =Image.open("/home/jupyter/images/"+im)
            if not check_min_size :
                check_min_size=(im.size)
            if im.size<check_min_size:
                check_min_size = im.size
        #print(check_min_size)
        return check_min_size

    def __len__(self):
        return len(self.data_example)


    def __getitem__(self, index):
        image_file_path, label = (self.data_example.images_id[index], self.data_example.label[index])#0 was index
        image = Image.open("/home/jupyter/images/{}{}".format(image_file_path,".jpg")).convert('RGB')
        image = self.transform(image)
        
        #features = self.pil_to_tensor(image)
        #features = self.resize(features)
        return image,label






In [263]:
from torch.utils.data import Dataset
import torch
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn 
import torch.backends.cudnn as cudnn #benchmark checks and selects the fastest conv algorithm
#from torchmetrics import Accuracy
import copy
import matplotlib as plt
from IPython.display import display, clear_output
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision.models import resnet50, ResNet50_Weights
import os
import datetime

class productImageClassifier(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        #self.resnet50 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50.fc = torch.nn.Linear(2048, 13)#this outputs numbers unormalized - we have 13 categories in the dictionary encoder 
        self.resnet50 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
        #print(self.resnet50.modules)
        
        for param in self.resnet50.parameters():
            param.requires_grad = False
        
        #eave this layer when training
        linear_layers = torch.nn.Sequential(
                        torch.nn.Linear(2048, 1050),
                        torch.nn.ReLU(),
                        torch.nn.Linear(1050, 500),
                        torch.nn.ReLU(),
                        torch.nn.Linear(500,2)
                        )
        #use the below layer for feature extraction 
        #linear_layers = torch.nn.Sequential(
        #                torch.nn.Linear(2048, 1000))
        
        self.resnet50.fc = linear_layers
        #self.resnet50.load_state_dict(torch.load('/home/jupyter/model_evaluation/model_04012023_153945/'))

    
    def forward(self, X):
        return self.resnet50(X)

def train(model, train_dataloader, val_dataloader, epochs=25):
    model.cuda()
    num_gpus=2
    model = torch.nn.parallel.DataParallel(model, device_ids=list(range(num_gpus)), dim=0)
    acc=0.0
    logs={"running_acc":[], "num_epochs": []}
    #optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    optimizer = optim.SGD(model.parameters(), lr=0.0009, momentum=0.9)
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)
    for epoch in range(1,epochs+1):
        print("Epoch: "+str(epoch))
        correct=0
        total=0
        model.train()
        for batch in train_dataloader:
            
            features, labels = batch
            features = features.cuda()
            labels = labels.cuda()

            with torch.set_grad_enabled(mode=True):
                predictions = model(features)
                labels = labels
                #labels = labels.float()
                #print(predictions.dtype)
                #print(labels.dtype)
                loss_model = nn.CrossEntropyLoss()
                loss = loss_model(predictions, labels)
                loss.backward()

                predictions = torch.argmax(predictions, dim=1)

            optimizer.step()
            optimizer.zero_grad()
            correct += (predictions == labels).float().sum()
            total+=features.size(0)
            #print("Nums correct:"+str(correct))
            #print("total count:"+str(total))
            
                
        #exp_lr_scheduler.step()
        print("Train running acc: "+str(100 * correct.item()/ total))

        with torch.no_grad():
            model.eval()
            correct_val=0
            total_val=0
            for batch_val in val_dataloader:
                features_val, labels_val =  batch_val
                features_val = features_val.cuda()
                labels_val = labels_val.cuda()
                predictions_val = model(features_val)#.unsqueeze(0))
                predictions_val = torch.argmax(predictions_val, dim=1)
                correct_val += (predictions_val == labels_val).float().sum()
                total_val +=features_val.size(0)
            print("Validation running acc: "+str(100 * correct_val.item()/ total_val))
        print("HERE")
        
        if not (os.path.exists("/home/jupyter/model_evaluation_2class")):
            os.mkdir("/home/jupyter/model_evaluation_2class")
        path = "/home/jupyter/model_evaluation_2class/model_{}".format(datetime.datetime.today().strftime('%m%d%Y_%H%M%S'))
        os.mkdir(path)
        torch.save(model.state_dict(), "{}/model-v3.pt".format(path))
       




        
                
    #plt.pyplot.plot(logs.running_acc,logs.num_epochs, 'r')  
    #plt.show() 


In [47]:
classifier_model =  productImageClassifier()

#check which layers are unfrozen
for name,para in classifier_model.named_parameters():
    if para.requires_grad == True:
        print(name)
        

    

resnet50.fc.0.weight
resnet50.fc.0.bias
resnet50.fc.2.weight
resnet50.fc.2.bias
resnet50.fc.4.weight
resnet50.fc.4.bias


In [48]:
#check which layers are unfrozen
for name,para in classifier_model.named_parameters():
    if para.requires_grad == True:
        print(name)

resnet50.fc.0.weight
resnet50.fc.0.bias
resnet50.fc.2.weight
resnet50.fc.2.bias
resnet50.fc.4.weight
resnet50.fc.4.bias


In [49]:
classifier_model =  productImageClassifier()
dataset = ProductsDataset()

train_dataset , val_dataset = random_split(dataset=dataset, lengths=[0.8,0.2], generator=torch.manual_seed(0))


In [50]:
#train
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=10, shuffle=True)
train(classifier_model, train_loader,val_loader)

#12604



Epoch: 1
Train running acc: 68.98550724637681
Validation running acc: 83.72093023255815
HERE
Epoch: 2
Train running acc: 83.33333333333333
Validation running acc: 90.40697674418605
HERE
Epoch: 3
Train running acc: 83.33333333333333
Validation running acc: 91.86046511627907
HERE
Epoch: 4
Train running acc: 82.31884057971014
Validation running acc: 93.6046511627907
HERE
Epoch: 5
Train running acc: 85.07246376811594
Validation running acc: 95.34883720930233
HERE
Epoch: 6
Train running acc: 85.8695652173913
Validation running acc: 93.8953488372093
HERE
Epoch: 7
Train running acc: 85.72463768115942
Validation running acc: 95.63953488372093
HERE
Epoch: 8
Train running acc: 87.10144927536231
Validation running acc: 94.47674418604652
HERE
Epoch: 9
Train running acc: 85.5072463768116
Validation running acc: 93.6046511627907
HERE
Epoch: 10
Train running acc: 86.66666666666667
Validation running acc: 94.18604651162791
HERE
Epoch: 11
Train running acc: 87.2463768115942
Validation running acc: 95.0

KeyboardInterrupt: 

In [68]:
#LOADING the model after using nn.paralell
#if we use nn.paralel we need to remove the last part of the model named module
# create new OrderedDict that does not contain `module.`
from collections import OrderedDict
new_state_dict = OrderedDict()
#print(state_dict.items())
for k, v in state_dict.items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# load params
model.load_state_dict(new_state_dict)



<All keys matched successfully>

In [83]:
#test with data
import numpy as np

prediction = model(dataset[1000][0].unsqueeze(0))
model.eval()
print(prediction)

print(torch.argmax(prediction))



tensor([[ 2.5893, -2.7020]], grad_fn=<AddmmBackward0>)
tensor(0)


In [352]:
#feature extraction
from collections import OrderedDict
import json
import numpy

class featExtraction(torch.nn.Module):
    def __init__(self) -> None:
        super().__init__()
        #self.model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_resnet50', pretrained=True)
        #self.resnet50.fc = torch.nn.Linear(2048, 13)#this outputs numbers unormalized - we have 13 categories in the dictionary encoder 
        self.model = productImageClassifier()
        self.model.eval()
        print(self.model.modules)

        self.featExtractionModel = self.load_trained_model()
        

            
            # Extract VGG-16 Average Pooling Layer
        self.pooling = self.featExtractionModel.resnet50.avgpool
        # Convert the image into one-dimensional vector
        self.flatten = nn.Flatten()
            # Replace fc layer with a linear layer with 1000 outputs
        self.fc = torch.nn.Linear(2048, 1000)
            #print(self.featExtractionModel.modules)

    def forward(self, X):
        #module = self.model._modules[self.target_layer]
        out= self.resnet50(X)
        #out = self.fc(X) 
        return  out


    def load_trained_model(self):
        state_dict = torch.load('/home/jupyter/model_evaluation_2class/model_04042023_205144/model-v3.pt') 

        new_state_dict = OrderedDict()
        #print(state_dict.items())
        for k, v in state_dict.items():
            name = k[7:] # remove `module.`
            new_state_dict[name] = v
        # load params
        self.model.load_state_dict(new_state_dict)
        return self.model.eval()
    
    def image_processing(self,path):
        image = Image.open("/home/jupyter/images/{}{}".format(path,".jpg")).convert('RGB')
        transform = transforms.Compose([
                transforms.Resize((224,224)),
                transforms.CenterCrop(224),
                #transforms.RandomResizedCrop(224),
                #transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        image = transform(image)
        return image.unsqueeze(0)
    
    def extract_feat(self, images_dataset):
        #img_features_dict = dict()
        images_list = images_dataset.images_id.to_list()
        #self.featExtractionModel.to(torch.device(1))
        for image in images_list:
            image_tensor = self.image_processing(image)
            img_features = self.featExtractionModel(image_tensor)
            #print(img_features.float())
            print(img_features.detach().numpy())
            img_features_dict = json.append({image: img_features.detach().numpy().tolist()})#img_features.detach().numpy()
        with open('/home/jupyter/img_embedings/images_embeddings.json', 'w') as fp:
            json.dump(img_features_dict, fp) 
            
        
        

In [353]:
#test model
dataset = ProductsDataset()
print(dataset[1000][0])


tensor([[[ 1.3242,  1.3242,  1.3070,  ...,  1.2728,  1.3927,  1.4269],
         [ 1.2214,  1.2557,  1.2728,  ...,  1.3584,  1.3584,  1.3413],
         [ 1.2557,  1.2557,  1.2899,  ...,  1.3070,  1.3584,  1.3927],
         ...,
         [-0.2684, -0.2342, -0.3027,  ...,  0.2453, -0.3027,  0.2282],
         [-0.2684, -0.3712, -0.2342,  ..., -0.2684,  0.1597,  0.5536],
         [-0.3369, -0.2342, -0.1828,  ..., -0.2342,  0.5022,  0.7248]],

        [[ 1.2031,  1.2031,  1.1856,  ...,  1.1506,  1.2731,  1.3081],
         [ 1.0980,  1.1331,  1.1506,  ...,  1.2381,  1.2381,  1.2206],
         [ 1.1331,  1.1331,  1.1681,  ...,  1.1856,  1.2381,  1.2731],
         ...,
         [-0.5651, -0.4776, -0.5651,  ..., -0.0574, -0.5651,  0.0476],
         [-0.5651, -0.6702, -0.5126,  ..., -0.5126, -0.0224,  0.3452],
         [-0.6352, -0.5301, -0.4776,  ..., -0.3901,  0.3102,  0.4678]],

        [[ 1.1411,  1.1411,  1.1237,  ...,  1.0888,  1.2108,  1.2457],
         [ 1.0365,  1.0714,  1.0888,  ...,  1

In [354]:
feat = featExtraction()


<bound method Module.modules of productImageClassifier(
  (resnet50): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=Tr

In [356]:
feat(dataset[1000][0].unsqueeze(0))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (672x224 and 2048x1050)

In [ ]:
feat.extract_feat(dataset.data_example)